# 데이터 살펴보기

In [7]:
# 필요한 라이브러리
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 불러오기
acc = pd.read_csv('accepted_2007_to_2018Q4 2.csv')

print(acc.shape)
acc.head()

/var/folders/3m/40zzfh3x6ns8ym348ltj6b640000gn/T/ipykernel_12903/4094139701.py:8: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  acc = pd.read_csv('accepted_2007_to_2018Q4 2.csv')


(2260701, 151)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 대출상환 항목 3개만 남기기
## acc = 대출상환목록 3개로 정리
acc= acc[acc['loan_status'].isin(['Fully Paid', 'Charged Off', 'Default'])].copy()

In [9]:
# 대출상환 항목 0,1로 인코딩
status_map = {
    'Fully Paid': 0,
    'Charged Off': 1,
    'Default': 1
}
acc['loan_status'] = acc['loan_status'].map(status_map)

In [ ]:
# null 값이 80% 이상인 컬럼 삭제
## acc = 대출상환목록 3개로 정리, Null 값이 80% 이상인 컬럼 삭제

threshold = acc.shape[0] * 0.8               # 기준 만들고
cols_to_drop = acc.columns[acc.isnull().sum() > threshold]  # 조건에 맞는 컬럼 추출
acc = acc.drop(columns=cols_to_drop)          # 해당 컬럼 삭제

In [ ]:
# 논문 참고하여 칼럼 셀렉
## acc = 대출상환목록 3개로 정리, 
#        Null 값이 80% 이상인 컬럼 삭제,
#        중요 칼럼만 남김,

columns_to_keep = [
    'loan_amnt', 'term', 'int_rate', 'installment', 'grade',
    'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
    'verification_status', 'purpose', 'dti', 'open_acc', 'revol_bal',
    'revol_util', 'total_acc', 'loan_status', 'pub_rec',
    'pub_rec_bankruptcies', 'mort_acc', 'inq_last_6mths',
    'inq_last_12m', 'num_bc_tl', 'num_actv_bc_tl'
]

acc = acc[columns_to_keep]

In [12]:
# 숫자형
acc.select_dtypes(include='number').columns

# 범주형
acc.select_dtypes(exclude='number').columns

Index(['term', 'grade', 'emp_title', 'emp_length', 'home_ownership',
       'verification_status', 'purpose'],
      dtype='object')

In [ ]:
# dti 의 이상치 개수 5473 _ IQR 사용 
# 이상치 제거 예정

Q1 = acc['dti'].quantile(0.25)
Q3 = acc['dti'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치만 추출
outliers_dti = acc[(acc['dti'] < lower_bound) | (acc['dti'] > upper_bound)]

print('이상치 개수: ' , len(outliers_dti))
print('해당 칼럼 전체 개수: ', len(acc.dti))
print('이상치 비율: ', round(len(outliers_dti) / len(acc.dti) * 100 , 2), '%' )


이상치 개수:  5473
해당 칼럼 전체 개수:  1345350
이상치 비율:  0.41 %


In [ ]:
# revol_util 이상치
# 이상치 제거 예정

Q1 = acc['revol_util'].quantile(0.25)
Q3 = acc['revol_util'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치만 추출
outliers_revol_util = acc[(acc['revol_util'] < lower_bound) | (acc['revol_util'] > upper_bound)]

print('이상치 개수: ' , len(outliers_revol_util))
print('해당 칼럼 전체 개수: ', len(acc.revol_util))
print('이상치 비율: ', round(len(outliers_revol_util) / len(acc.revol_util) * 100 , 2), '%' )


이상치 개수:  72
해당 칼럼 전체 개수:  1345350
이상치 비율:  0.01 %


In [ ]:
# pub_rec_bankruptcies(파산 기록수) 이상치
# 이상치 편집 예정

Q1 = acc['pub_rec_bankruptcies'].quantile(0.25)
Q3 = acc['pub_rec_bankruptcies'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치만 추출
outliers_rec_bnkrcys = acc[(acc['pub_rec_bankruptcies'] < lower_bound) | (acc['pub_rec_bankruptcies'] > upper_bound)]

print('이상치 개수: ' , len(outliers_rec_bnkrcys))
print('해당 칼럼 전체 개수: ', len(acc.pub_rec_bankruptcies))
print('이상치 비율: ', round(len(outliers_rec_bnkrcys) / len(acc.pub_rec_bankruptcies) * 100 , 2), '%' )

이상치 개수:  167666
해당 칼럼 전체 개수:  1345350
이상치 비율:  12.46 %


In [ ]:
# mort_acc 이상치
# 이상치 제거 예정

Q1 = acc['mort_acc'].quantile(0.25)
Q3 = acc['mort_acc'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치만 추출
outliers_mort_acc = acc[(acc['mort_acc'] < lower_bound) | (acc['mort_acc'] > upper_bound)]

print('이상치 개수: ' , len(outliers_mort_acc))
print('해당 칼럼 전체 개수: ', len(acc.mort_acc))
print('이상치 비율: ', round(len(outliers_mort_acc) / len(acc.mort_acc) * 100 , 2), '%' )

이상치 개수:  17794
해당 칼럼 전체 개수:  1345350
이상치 비율:  1.32 %


In [ ]:
# inq_last_12m 이상치
# 이상치 제거 예정

Q1 = acc['inq_last_12m'].quantile(0.25)
Q3 = acc['inq_last_12m'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치만 추출
outliers_inq_last_12m = acc[(acc['inq_last_12m'] < lower_bound) | (acc['inq_last_12m'] > upper_bound)]

print('이상치 개수: ' , len(outliers_inq_last_12m))
print('해당 칼럼 전체 개수: ', len(acc.inq_last_12m))
print('이상치 비율: ', round(len(outliers_inq_last_12m) / len(acc.inq_last_12m) * 100 , 2), '%' )

이상치 개수:  34344
해당 칼럼 전체 개수:  1345350
이상치 비율:  2.55 %


In [ ]:
# num_bc_tl 이상치
# 이상치 제거 예정

Q1 = acc['num_bc_tl'].quantile(0.25)
Q3 = acc['num_bc_tl'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치만 추출
outliers_num_bc_tl = acc[(acc['num_bc_tl'] < lower_bound) | (acc['num_bc_tl'] > upper_bound)]

print('이상치 개수: ' , len(outliers_num_bc_tl))
print('해당 칼럼 전체 개수: ', len(acc.num_bc_tl))
print('이상치 비율: ', round(len(outliers_num_bc_tl) / len(acc.num_bc_tl) * 100 , 2), '%' )

이상치 개수:  27144
해당 칼럼 전체 개수:  1345350
이상치 비율:  2.02 %


In [ ]:
# num_actv_bc_tl 이상치
# 이상치 제거 예정

Q1 = acc['num_actv_bc_tl'].quantile(0.25)
Q3 = acc['num_actv_bc_tl'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치만 추출
outliers_num_actv_bc_tl = acc[(acc['num_actv_bc_tl'] < lower_bound) | (acc['num_actv_bc_tl'] > upper_bound)]

print('이상치 개수: ' , len(outliers_num_actv_bc_tl))
print('해당 칼럼 전체 개수: ', len(acc.num_actv_bc_tl))
print('이상치 비율: ', round(len(outliers_num_actv_bc_tl) / len(acc.num_actv_bc_tl) * 100 , 2), '%' )

이상치 개수:  26648
해당 칼럼 전체 개수:  1345350
이상치 비율:  1.98 %


In [60]:
# 이상치 지울 데이터 따로 만들기
## acc_ver2 = 대출상환목록 3개로 정리, 
#             Null 값이 80% 이상인 컬럼 삭제,
#             중요 칼럼만 남김,
acc_ver2 = acc.copy()

In [61]:
### ver2 데이터에서 이상치 모두 제거하기

## acc_ver2 = 대출상환목록 3개로 정리, 
#             Null 값이 80% 이상인 컬럼 삭제,
#             중요 칼럼만 남김,

## acc_cleaned = 대출상환목록 3개로 정리, 
#                Null 값이 80% 이상인 컬럼 삭제,
#                중요 칼럼만 남김,
#                미미한 이상치들 제거


# 이상치 하나로 모으기                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
outliers_all = [outliers_dti,
    outliers_revol_util,
    outliers_mort_acc,
    outliers_inq_last_12m,
    outliers_num_bc_tl,
    outliers_num_actv_bc_tl
]

# 이상치 데이터 하나로 합치기
outliers_all = pd.concat(outliers_all)

# 중복된 인덱스 제거 (선택사항이지만 코드 정돈을 위해)
outliers_all = outliers_all[~outliers_all.index.duplicated(keep='first')]

# 기존 데이터(acc_ver2)에서 이상치 제거한 새 데이터 만들기
acc_cleaned = acc_ver2.drop(index=outliers_all.index).reset_index(drop=True)

# 이상치 제거 후 null 값 개수 확인
acc.select_dtypes(include='number').isnull().sum()



loan_amnt                    0
int_rate                     0
installment                  0
annual_inc                   0
dti                        374
open_acc                     0
revol_bal                    0
revol_util                 857
total_acc                    0
loan_status                  0
pub_rec                      0
pub_rec_bankruptcies       697
mort_acc                 47281
inq_last_6mths               1
inq_last_12m            807715
num_bc_tl                67527
num_actv_bc_tl           67527
dtype: int64

In [70]:
dti_rate = round(acc_cleaned.dti.isnull().sum() / acc_cleaned.dti.shape[0] * 100 ,5)
print('dti null 값 비율: ' , dti_rate)
revol_util_rate = round(acc_cleaned.revol_util.isnull().sum() / acc_cleaned.revol_util.shape[0] * 100 ,5)
print('revol_util null 값 비율: ' , revol_util_rate)
pub_rec_bankruptcies_rate = round(acc_cleaned.pub_rec_bankruptcies.isnull().sum() / acc_cleaned.pub_rec_bankruptcies.shape[0] * 100 ,5)
print('pub_rec_bankruptcies null 값 비율: ' , pub_rec_bankruptcies_rate)
mort_acc_rate = round(acc_cleaned.mort_acc.isnull().sum() / acc_cleaned.mort_acc.shape[0] * 100 ,5)
print('mort_acc null 값 비율: ' , mort_acc_rate)
inq_last_12m_rate = round(acc_cleaned.inq_last_12m.isnull().sum() / acc_cleaned.inq_last_12m.shape[0] * 100 ,5)
print('inq_last_12m null 값 비율: ' , inq_last_12m_rate)
num_bc_tl_rate = round(acc_cleaned.num_bc_tl.isnull().sum() / acc_cleaned.num_bc_tl.shape[0] * 100 ,5)
print('num_bc_tl null 값 비율: ' , num_bc_tl_rate)
num_actv_bc_tl_rate = round(acc_cleaned.num_actv_bc_tl.isnull().sum() / acc_cleaned.num_actv_bc_tl.shape[0] * 100 ,5)
print('num_actv_bc_tl null 값 비율: ' , num_actv_bc_tl_rate)

dti null 값 비율:  0.02834
revol_util null 값 비율:  0.06584
pub_rec_bankruptcies null 값 비율:  0.05596
mort_acc null 값 비율:  3.79627
inq_last_12m null 값 비율:  61.66024
num_bc_tl null 값 비율:  5.3887
num_actv_bc_tl null 값 비율:  5.3887


In [78]:
# pub_rec  == 0 이고 pub_rec_bankruptcies == Null 인 항목 개수

aa = acc_cleaned.loc[(acc_cleaned.pub_rec == 0) & (acc_cleaned.pub_rec_bankruptcies.isnull())]

aa.shape[0]

677

In [ ]:
# pub_rec  == 0 이고 pub_rec_bankruptcies == Null 인 항목들 pub_rec_bankruptcies 칼럼에서 0으로 만들기

acc_cleaned.loc[
    (acc_cleaned['pub_rec'] == 0) & (acc_cleaned['pub_rec_bankruptcies'].isnull()),
    'pub_rec_bankruptcies'
] = 0


In [83]:
# 이후 null 값 개수 확인 
 
acc_cleaned.pub_rec_bankruptcies.isnull().sum()

np.int64(20)

In [84]:
# dti NA들이 연간 소득 0인 거랑 연관있는지 확인
print(acc_cleaned[acc_cleaned['annual_inc']==0].shape[0])
print(acc_cleaned.loc[acc_cleaned['annual_inc']==0, 'dti'].unique())
print(acc_cleaned[(acc_cleaned['annual_inc'] == 0) & (acc_cleaned['verification_status'] == 'Not Verified')].shape[0])
# 관련 존재! 연간 소득 0이면 dti NA임
acc_cleaned[(acc_cleaned['annual_inc']!=0) & (acc_cleaned['dti'].isnull())] # 연간 소득이 1 or 2
# 351

342
[nan]
319


,loan_amnt,term,int_rate,installment,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,...,revol_util,total_acc,loan_status,pub_rec,pub_rec_bankruptcies,mort_acc,inq_last_6mths,inq_last_12m,num_bc_tl,num_actv_bc_tl
371239,6000.0,36 months,10.42,194.79,B,NaN,NaN,MORTGAGE,1.0,Not Verified,...,20.4,17.0,0,0.0,0.0,2.0,0.0,2.0,4.0,3.0
374169,40000.0,60 months,12.62,902.37,C,NaN,NaN,MORTGAGE,1.0,Not Verified,...,72.7,24.0,0,0.0,0.0,4.0,0.0,0.0,6.0,4.0
377231,11000.0,36 months,10.91,359.66,B,NaN,NaN,MORTGAGE,1.0,Not Verified,...,34.3,15.0,0,0.0,0.0,0.0,0.0,1.0,4.0,3.0
449882,2400.0,36 months,11.47,79.11,B,NaN,NaN,RENT,1.0,Not Verified,...,53.8,21.0,1,0.0,0.0,0.0,1.0,1.0,7.0,3.0
536730,24000.0,36 months,8.39,756.40,B,NaN,NaN,MORTGAGE,1.0,Not Verified,...,54.8,11.0,0,0.0,0.0,1.0,0.0,1.0,7.0,3.0
565842,18000.0,60 months,16.29,440.51,D,NaN,NaN,MORTGAGE,1.0,Not Verified,...,31.9,23.0,1,1.0,1.0,1.0,0.0,2.0,8.0,2.0
836963,8000.0,36 months,25.81,321.52,E,NaN,NaN,MORTGAGE,1.0,Not Verified,...,73.5,28.0,1,0.0,0.0,1.0,0.0,1.0,8.0,2.0
1175584,15000.0,36 months,10.91,490.45,B,RN,10+ years,RENT,2.0,Not Verified,...,67.6,24.0,0,0.0,0.0,3.0,1.0,1.0,7.0,1.0
1190155,3600.0,36 months,14.08,123.18,C,NaN,NaN,RENT,1.0,Verified,...,19.2,16.0,0,0.0,0.0,0.0,1.0,1.0,6.0,3.0
1193187,40000.0,60 months,9.93,848.51,B,Admin,< 1 year,MORTGAGE,1.0,Not Verified,...,24.3,11.0,0,0.0,0.0,0.0,1.0,1.0,7.0,4.0


In [85]:
# # 리볼빙 신용 총 한도 - total_rev_hi_lim 칼럼 있어야 확인 가능
# print(acc_cleaned[(acc_cleaned['total_rev_hi_lim'] == 0) | (acc_cleaned['total_rev_hi_lim'].isnull())].shape[0])
# print('리볼빙 신용 총 한도가 0인 데이터 수', acc_cleaned[(acc_cleaned['total_rev_hi_lim'] == 0)].shape[0], '\n 총 한도가 na인 데이터 수' , acc_cleaned[acc_cleaned['total_rev_hi_lim'].isnull()].shape[0])
# # 리볼빙 신용 총 한도 == 0 이면서 revol_util NA
# print(acc_cleaned[(acc_cleaned['total_rev_hi_lim'] == 0) & (acc_cleaned['revol_util'].isnull())].shape[0])
# print(acc_cleaned[(acc_cleaned['total_rev_hi_lim'].isnull()) & (acc_cleaned['revol_util'].isnull())].shape[0])
# acc_cleaned.loc[((acc_cleaned['total_rev_hi_lim'] != 0 ) & (acc_cleaned['total_rev_hi_lim'].notna())) & (acc_cleaned['revol_util'].isnull()), 'revol_util':'total_rev_hi_lim']

In [87]:
# 파산 기록 NA 값 확인
print(acc_cleaned[acc_cleaned['pub_rec_bankruptcies'] >= 0].shape[0])
print(acc_cleaned.loc[(acc_cleaned['pub_rec'].notnull()) & (acc_cleaned['pub_rec_bankruptcies'].isnull()), ['pub_rec', 'pub_rec_bankruptcies', 'verification_status']])
print(acc_cleaned.loc[(acc_cleaned['pub_rec'] == 0) & (acc_cleaned['pub_rec_bankruptcies'].isnull())].shape[0])
print(acc_cleaned.loc[(acc_cleaned['pub_rec'] == 0) & (acc_cleaned['pub_rec_bankruptcies'].isnull()), ['pub_rec', 'pub_rec_bankruptcies', 'verification_status']].value_counts('verification_status'))
print(acc_cleaned.loc[(acc_cleaned['pub_rec'] != 0) & (acc_cleaned['pub_rec_bankruptcies'].isnull())])

1245439
        pub_rec  pub_rec_bankruptcies verification_status
888247      1.0                   NaN        Not Verified
888261      1.0                   NaN        Not Verified
888286      1.0                   NaN        Not Verified
888322      1.0                   NaN        Not Verified
888340      1.0                   NaN        Not Verified
888351      1.0                   NaN        Not Verified
888446      1.0                   NaN        Not Verified
888465      1.0                   NaN        Not Verified
888510      1.0                   NaN        Not Verified
888536      1.0                   NaN        Not Verified
888567      1.0                   NaN        Not Verified
888629      1.0                   NaN        Not Verified
888640      1.0                   NaN        Not Verified
888676      1.0                   NaN        Not Verified
888677      1.0                   NaN        Not Verified
888741      2.0                   NaN        Not Verified
888747

In [88]:
# mort_acc - 모기지 대출 계좌수
print(acc_cleaned[acc_cleaned['mort_acc'] >= 0].shape[0]) # 0 존재
print(acc_cleaned[(acc_cleaned['home_ownership'] == 'NONE') & (acc_cleaned['mort_acc'].isnull())])
print(acc_cleaned[(acc_cleaned['home_ownership'] == 'MORTGAGE') & (acc_cleaned['mort_acc'].isnull())].shape[0])
print(acc_cleaned[(acc_cleaned['home_ownership'] == 'MORTGAGE') & (acc_cleaned['mort_acc'].isnull())].value_counts(['verification_status']))
print(acc_cleaned[(acc_cleaned['home_ownership'].isin(['RENT', 'OWN', 'ANY', 'OTHER'])) & (acc_cleaned['mort_acc'].isnull())].value_counts(['home_ownership', 'verification_status']))


1198178
        loan_amnt        term  int_rate  installment grade emp_title  \
888550    10000.0   36 months      7.75       312.22     A       ups   
888891     4000.0   36 months      9.64       128.41     B       NaN   
888892     2800.0   36 months      8.70        88.65     B       NaN   

       emp_length home_ownership  annual_inc verification_status  ...  \
888550    5 years           NONE     22200.0        Not Verified  ...   
888891   < 1 year           NONE    100000.0        Not Verified  ...   
888892   < 1 year           NONE    120000.0        Not Verified  ...   

       revol_util  total_acc  loan_status  pub_rec  pub_rec_bankruptcies  \
888550        8.7       22.0            0      0.0                   0.0   
888891       14.9        8.0            0      0.0                   0.0   
888892       14.9        8.0            0      0.0                   0.0   

        mort_acc  inq_last_6mths  inq_last_12m  num_bc_tl  num_actv_bc_tl  
888550       NaN             

In [89]:
acc_cleaned.groupby('home_ownership')['mort_acc'].agg(['mean', 'median'])

,mean,median
home_ownership,,
ANY,1.459459,1.0
MORTGAGE,2.472820,2.0
NONE,1.720930,1.0
OTHER,1.534884,1.0
OWN,1.225130,1.0
RENT,0.512155,0.0


In [90]:
# inq_last_6mths 지난 6개월간 신용조회 횟수
# na 값있는 행 확인
print(acc_cleaned[acc_cleaned['inq_last_6mths'].isna()])
print(acc_cleaned[acc_cleaned['inq_last_6mths'] == 0].shape[0])

        loan_amnt        term  int_rate  installment grade  \
603969    35000.0   60 months       9.8       740.21     B   

                   emp_title emp_length home_ownership  annual_inc  \
603969  Subcontracts Manager  10+ years       MORTGAGE    110000.0   

       verification_status  ... revol_util  total_acc  loan_status  pub_rec  \
603969     Source Verified  ...        3.5       36.0            0      0.0   

        pub_rec_bankruptcies  mort_acc  inq_last_6mths  inq_last_12m  \
603969                   0.0       4.0             NaN           NaN   

        num_bc_tl  num_actv_bc_tl  
603969       10.0             1.0  

[1 rows x 24 columns]
729705


In [91]:
acc_cleaned['inq_last_6mths'].agg(['mean', 'median'])

mean      0.621508
median    0.000000
Name: inq_last_6mths, dtype: float64

In [92]:
acc_cleaned.loc[acc_cleaned['grade'] == 'B'].groupby(['grade', 'home_ownership', 'verification_status'])['inq_last_6mths'].agg(['mean', 'median'])

mean  median
grade home_ownership verification_status                  
B     ANY            Not Verified         0.323529     0.0
                     Source Verified      0.421053     0.0
                     Verified             0.533333     0.0
      MORTGAGE       Not Verified         0.527686     0.0
                     Source Verified      0.522616     0.0
                     Verified             0.607652     0.0
      NONE           Not Verified         0.666667     0.5
                     Verified             0.714286     0.0
      OTHER          Not Verified         0.863636     1.0
                     Source Verified      0.800000     1.0
                     Verified             1.200000     1.0
      OWN            Not Verified         0.496880     0.0
                     Source Verified      0.487075     0.0
                     Verified             0.519481     0.0
      RENT           Not Verified         0.466111     0.0
                     Source Verified      0.454960     0.0
                     Verified             0.471548     0.0

In [93]:
# num_bc_tl & num_actv_bc_tl
# 총 신용카드 계좌 수, 활성화 된 신용카드 계좌 수
print(acc_cleaned[acc_cleaned['num_bc_tl'].isnull() & acc_cleaned['num_actv_bc_tl'].isnull()].shape[0])
print(acc_cleaned[acc_cleaned['num_bc_tl'] == 0].shape[0])
print(acc_cleaned[acc_cleaned['num_bc_tl'].isnull() & acc_cleaned['num_actv_bc_tl'].isnull() & acc_cleaned['mort_acc'].isnull()
            & acc_cleaned['inq_last_12m'].isnull()].shape[0])

67114
2798
47281


In [94]:
print(acc_cleaned['num_actv_bc_tl'].agg(['mean', 'median']))
print(acc_cleaned['num_bc_tl'].agg(['mean', 'median']))

mean      3.439613
median    3.000000
Name: num_actv_bc_tl, dtype: float64
mean      7.59055
median    7.00000
Name: num_bc_tl, dtype: float64


# 데이터 전처리

* dti null 값 비율:  0.02893
* revol_util null 값 비율:  0.07387
* pub_rec_bankruptcies null 값 비율:  0.0638
* mort_acc null 값 비율:  4.15554
* inq_last_12m null 값 비율:  62.41306
* num_bc_tl null 값 비율:  5.92643
* num_actv_bc_tl null 값 비율:  5.92643

In [95]:
acc_cleaned.isnull().sum()

loan_amnt                    0
term                         0
int_rate                     0
installment                  0
grade                        0
emp_title                78075
emp_length               71243
home_ownership               0
annual_inc                   0
verification_status          0
purpose                      0
dti                        353
open_acc                     0
revol_bal                    0
revol_util                 820
total_acc                    0
loan_status                  0
pub_rec                      0
pub_rec_bankruptcies        20
mort_acc                 47281
inq_last_6mths               1
inq_last_12m            767953
num_bc_tl                67114
num_actv_bc_tl           67114
dtype: int64

In [97]:
# 이상치 미미한 항목들 해당 칼럼에서 dropna 실행

acc_cleaned = acc_cleaned.dropna(subset = ['dti', 'revol_util', 'inq_last_6mths'], axis=0)